In [68]:
import pandas as pd

In [69]:
columns = ["timestamp", "role", "years_experience", "programming_language", "programming_packages", "data_tools", "cloud_provider"]
survey_df = pd.read_csv("../data/data_tools_survey_results.csv", names=columns, skiprows=1)
# do not care about the timestamp column
survey_df = survey_df.drop(columns="timestamp")
survey_df.head()

,role,years_experience,programming_language,programming_packages,data_tools,cloud_provider
0,Data Analytics/Business Intelligence,5+ years,"Python, R, SQL","pandas, matplotlb, seaborn","Tableau, Visual Studio Code, Atom, DBeaver",Amazon Web Services
1,Data Engineering,5+ years,"R, SQL",Tidyverse,"Tableau, Excel, Alteryx, RStudio",Microsoft Azure
2,Data Science,5+ years,Python,Jupyter notebooks,"Tableau, Excel, Juicebox",Amazon Web Services
3,Data Science,0 to 2 years,"R, SQL, SAS, STATA",dplyr,NaN,SAS sever
4,Data Science,5+ years,"Python, SQL","Spacy,gensim,pandas,sqlalchemy","PyCharm, Jupyer",Amazon Web Services


In [70]:
role_map_df = pd.read_csv("../mapping/role_mapping.csv")
role_map_df.head()

,orig_role,mapped_role
0,"Search Relevance, so it's hard to classify",Other
1,Machine learning research; Science that genera...,Other
2,Salesforce development,Other
3,Programming / Developer work,Other
4,Business management,Other


In [71]:
survey_df.shape

(26, 6)

In [72]:
survey_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26 entries, 0 to 25
Data columns (total 6 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   role                  26 non-null     object
 1   years_experience      26 non-null     object
 2   programming_language  25 non-null     object
 3   programming_packages  19 non-null     object
 4   data_tools            25 non-null     object
 5   cloud_provider        23 non-null     object
dtypes: object(6)
memory usage: 1.3+ KB


In [73]:
def map_values_for_column(df, mapped_df):
    for index, row in mapped_df.iterrows():
        df["role"] = df["role"].replace([row["orig_role"]], row["mapped_role"])

    return df

In [74]:
survey_df = map_values_for_column(survey_df, role_map_df)
survey_df["role"].unique()

array(['Data Analytics/Business Intelligence', 'Data Engineering',
       'Data Science', 'Other', 'Machine Learning Engineering'],
      dtype=object)